## Hidden Markov Model Study

Goal: Revisit Hidden Markov Models from the start, write slow but readable code. Build a Hidden Markov Model from scratch.

Stretch goals: 
- revisit Theoretic foundations for Multivariate Categorically distributed emissions.
- how does the independence assumptions for multivariate emissions change the theory?


## In the Defense of Simple Models

Motivating Hidden Markov Models is hard without asking some fundamental questions about statistical and probabilistic inference and learning.

Why do we model things? What are we interested about finding out - the underlying dynamics or solely predictions into the future? Do we even want to predict into the future or are we satisfied with estimating some underlying state responsible for our observations?
Why not choose Timeseries model type X over type Y? Well, there is no easy answer to this, it may just be that another type of model is better fit for a specific requirement.

Markov Models and *Hidden* Markov Models are so-called **generative** models. This means that after building them through a process called *training* or *fitting*, we may produce new pseudo-data from our model. We may extrapolate from timepoint $t$ onwards or simulate a new trajectory. We may choose different initial conditions and see, how the model evolves. It is here, i want to make an argument for Hidden Markov Models in the age of Neural Networks. 
The biggest advantage of HMMs over Neural Methods is in my understanding, that they are **interpretable**. They are white-box models, allowing the observer to clearly understand **why** the model has produced any output. To some degree, the data **is** the model. All important information has been distilled and transformed into a model.

Another big advantage is that they are **simple** models. No-hyperparamter-tuning. No tinkering with the learning rate or swapping out optimizers, adding dropout, adding layer norms, removing layer norms... The process of fitting the model is more sane. In some sense this is a half-truth, as we do have to make assumptions when we choose a model type, however they are very different types of assumptions. They encourage us to think harder about the problem and the setup, not decide on model changes depending on whether or not the $F_1$-Score is moving up or down.

## Definitions.

Let a Hidden Markov Model be a tuple $\lambda = (\mathcal A, \mathcal B, \pi)$ where 

\begin{align*}
    \mathcal A := p(x_{t}|X_{0:t-1})\quad & \text{Conditional distribution over hidden state given previous hidden states}\\
    \mathcal B := p(y_{t}|X_{0:t})\quad& \text{Conditional distribution over emission signal given current and previous hidden states}\\ 
    \pi := p(x_0)\quad & \text{Initial state distribution}
\end{align*}

In my thesis and the following study, we will look at a special case of HMMs, where $\mathcal A, \mathcal B$ and $\pi$ are categorical distributions, making them row-stochastic matrices containing transition and emission probabilities. The probabilistic formulation of the object lets us see however, how we might use **Bayes-Rule** to do inference on the hidden states, as

$$
    p(X_{0:t}|y_t) = \frac{p(y_t|X_{0:t})p(X_{0:t})}{p(y_t)} \quad \text{ or } \quad p(X_{0:N}| Y_{0:N}) = \frac{p(Y_{0:N}|X_{0:N})p(X_{0:N})}{p(Y_{0:N})}
$$

Calculating these expressions however is untractable, as it would require us to compute large integrals $p(Y) = \int p(Y|X)p(X) dX$ where $p(X) = p(x_0)p(x_1|x_0)\dots p(x_N|x_0, \dots, x_{N-1})$ itself would be tricky to compute. It is here where we make **simplifying independence assumptions** called the *Markov Assumption*:

$$
    p(x_t|x_0, x_1, \dots x_{t-1}) = p(x_t|x_{t-1}) \quad \text{and} \quad p(y_t|x_0, x_1, \dots, x_{t}) = p(y_t|x_t)
$$

In words, we assume that transitions only depend on the previously visited state and that emissions only depend on the current hidden state. This simplifies the computations, as we may factor expressions as $p(X)$ into parts. 



### 

## Introductory Example

We are concerned with making statements about any **hidden** or **latent** variable $x$. In this example, lets consider the following setup: You're sitting at home, listening to the weather news on the radio. The true weather however is not-observable, as your window blinds are down. In our model, the true weather will be the non-observable (i.e. the *hidden* variable $x_t$) and the news will be the observable variable $y_t$. There are only two different 'true' weather states: Sunny $S$ and Rainy $R$, whereas the media reports three different observations: Dry $D$, Wet $W$ and Cloudy $C$.
The prior over the hidden states is a discrete uniform distribution. Additionally, we **know** $\mathcal A, \mathcal B$. 

**IMPORTANT** This is obviously a shortcut. In reality we dont know $\mathcal A$ and $\mathcal B$, we have to fit them, using our data! Notice, how our motivation has changed the model and how we use it. Instead of being interested *how exactly* the weather changes or in which hidden state what wheather observation is made, we focus on prediction rather than trying to understand underlying dynamics.

We summarize the assumptions with these equations:

\begin{align*}
    \mathcal A &= \begin{bmatrix}
        P(S|S) & P(R|S)\\
        P(S|R) & P(R|R)
    \end{bmatrix} = \begin{bmatrix}
        0.7 & 0.3\\
        0.2 & 0.8
    \end{bmatrix}\\
    \mathcal B &= \begin{bmatrix}
        P(D|S) & P(W|S) & P(C|S)\\
        P(D|R) & P(W|R) & P(C|R)
    \end{bmatrix} = \begin{bmatrix}
        0.5 & 0.3 & 0.2\\
        0.0 & 0.8 & 0.2
    \end{bmatrix}\\
    \pi &= \begin{bmatrix}
        P(x_0 = S)\\
        P(x_0 = R)
    \end{bmatrix} = \begin{bmatrix}
        0.5\\
        0.5
    \end{bmatrix}
\end{align*}

### Predicting and Updating

Inference on timeseries is usually separated into three steps: **predicting** the next hidden state $\tilde x = p(x_t| X_{0:t-1}, Y_{0:t-1})$ and **updating** said prediction once we have observed the current emission $\hat x = p(x_t | X_{0:t-1}, Y_{0:t-1}, y_t)$

The recursive prediction equation is:

$$
    p(x_t| Y_{0:t-1}) = \int \underbrace{p(x_t|x_{t-1})}_{\text{hidden state dynamics}}\cdot \overbrace{p(x_{t-1}|Y_{0:t-1})}^{\text{previous update result} } dx_{t-1}  
$$

This is a local integral over $x_{t-1}$, which we may be able to solve, if we have access to the previous update result, which is similarily recursively defined as:

$$
    p(x_t|Y_{0:t}) = \frac{\overbrace{p(y_t|x_t)}^{\text{likelihood}}\overbrace{p(x_t|Y_{0:t-1})}^{\quad \text{prev. prediction result}}}{\int p(y_t | x_t )p(x_t | Y_{0:t−1}) dx_t}
$$

We can see, that we have to compute the prediction and the update recursively, in order to get an updated estimate of the distribution over $x_t$.

### Calculation

Now, how would we go about calculating these quantities? We should start at the beginning, calculating the prediction of $x_1$, before updating it with our observation. For the sake of this example, lets assume we've observed the sequence $Y_{0:3} = \{ D, C, C, W\}$ and the state and emission matrices are as described above.

Calculate the update for $t=0$:

\begin{align*}
    p(x_0) &= \pi\\
    p(x_0|y_0) &= \frac{p(y_0|x_0)p(x_0)}{\int p(y_0|x_0)p(x_0) dx_0} = \frac{\begin{bmatrix}
        P(D|S)\\
        P(D|R)
    \end{bmatrix}\odot \begin{bmatrix}
        P(S)\\
        P(R)
    \end{bmatrix}}{P(D|S)P(S) + P(D|R)P(R)}\\
    &= \frac{\begin{bmatrix}
        0.5\\
        0.0
    \end{bmatrix}\odot \begin{bmatrix}
        0.5\\
        0.5
    \end{bmatrix}}{0.5 \cdot 0.5 + 0.0 \cdot 0.5} = \begin{bmatrix}
        1\\
        0
    \end{bmatrix}
\end{align*}\\


Seems like after seeing the first observation $y_0 = D$, we are completely convinced, that it is sunny outside. Lets see, what happens, if we predict the next hidden state.

\begin{align*}
    p(x_0|y_0) &= \begin{bmatrix}
        1\\
        0
    \end{bmatrix}\\
    p(x_1|x_0, y_0) &= \int p(x_1|x_0)p(x_0|y_0)dx_0\\
    &= \mathcal A ^T p(x_0|y_0)\\
    &= \begin{bmatrix}
        0.7\\
        0.3
    \end{bmatrix}
\end{align*}

Finally, lets update for our estimate of the hidden state at $t=1$. For this, we calculate the update analogously to our update at timestep $t= 0$, with the difference, that now we use the posterior of the prediction as the prior.

\begin{align*}
    p(x_1) &= p(x_1|y_0, x_0) = \begin{bmatrix}
    0.7\\
    0.3
    \end{bmatrix}\\
    p(x_1|Y_{0:1}) &= \frac{\mathcal B[:, 2] \odot p(x_1|y_0, x_0)}{\sum_i \mathcal B[:, 2] \odot p(x_1|y_0, x_0)}\\
    &= \frac{\begin{bmatrix}
        0.2\\
        0.2
    \end{bmatrix}\odot \begin{bmatrix}
        0.7\\
        0.3
    \end{bmatrix}}{0.2 \cdot 0.7 + 0.2 \cdot 0.3}\\
    &= \begin{bmatrix}
        0.7\\
        0.3
    \end{bmatrix}
\end{align*}\\



We will put this prediction and update routine in code below:

In [1]:
import numpy as np

A = np.array([
    [0.7, 0.3],
    [0.2, 0.8]
])

B = np.array([
    [0.5, 0.2, 0.3],
    [0.0, 0.2, 0.8]
])

x_0 = np.array([0.5, 0.5])

In [2]:
def update(x_tm1 : np.ndarray, y_t : int,  B : np.ndarray) -> np.ndarray:

    likelihood = B[:, y_t]
    prior = x_tm1

    posterior_unnormalized = likelihood * prior
    return posterior_unnormalized / posterior_unnormalized.sum()

def predict(x_tm1 : np.ndarray, A : np.ndarray) -> np.ndarray:

    # check for row-stochasticity
    assert np.all(np.isclose(A.sum(axis=1), 1)), "Matrix A doesn't seem to be row-stochastic"

    #check if x_tm1 is a valid probability distribution
    assert np.isclose(x_tm1.sum(), 1), "x_tm1 isn't stochastic, sum != 1"

    return A.T @ np.atleast_1d(x_tm1)


In [3]:
observations = [0, 1, 1, 2] # [D, C, C, W]
x_tm1 = x_0
predictions = [x_0]
updates = []

for y_i in observations:
    x_updated = update(x_tm1, y_i, B)
    x_tm1 = predict(x_updated, A)

    updates.append(x_updated)
    predictions.append(x_tm1)

    
predictions, updates

([array([0.5, 0.5]),
  array([0.7, 0.3]),
  array([0.55, 0.45]),
  array([0.475, 0.525]),
  array([0.32666667, 0.67333333])],
 [array([1., 0.]),
  array([0.7, 0.3]),
  array([0.55, 0.45]),
  array([0.25333333, 0.74666667])])

### Multivariate Case

Before looking at smoothing, let us think about changing the likelihood function $p(y_t | x_t)$. What would change, if y_t would be a collection of $K$ independent categorically distributed random variables instead of a single random variable? We would have to keep track of $K$ different emission matrices and calculate the update differently, as $p(y_t|x_t)$ now expands into $\prod_{i = 1}^K p(y^i_t | x_t)$.

We extend our example of the TV and weather with another observation variable. Lets assume we would observe dogs barking depending on the weather. If it's sunny, people much rather go out to walk their dog than if its raining. From this assumption follows the emission matrix $\mathcal B_2 = \begin{bmatrix}
    P(B|S) & P(\neg B|S)\\
    P(B|R) & P(\neg B|R)
\end{bmatrix} = \begin{bmatrix}
    0.6 & 0.4\\
    0.1 & 0.9
\end{bmatrix}$

In [4]:
A = np.array([
    [0.7, 0.3],
    [0.2, 0.8]
])

B_1 = np.array([
    [0.5, 0.2, 0.3],
    [0.0, 0.2, 0.8]
])

B_2 = np.array([
    [0.6, 0.4],
    [0.1, 0.9]
])

x_0 = np.array([0.5, 0.5])

In [5]:
from typing import Sequence

In [6]:
def update_mv(x_tm1 : np.ndarray, y_t : Sequence[int],  Bs : Sequence[np.ndarray]) -> np.ndarray:

    """
        x_tm1   : np.ndarray        vector of hidden state distribution from timestep t-1
        y_t     : Sequence[int]     Sequence of ints, int at index i contains observation for corresponding emission matrix at index i
        Bs      : Sequence[np.ndarray]  Sequence of emission matrices              
    """

    # likelihood p(y_t | x_t) factorizes into p(y^1_t|x_t) * p(y^2_t|x_t) * ... *  p(y^K_t|x_t)
    likelihoods = np.ones_like(x_tm1)
    for y, B in zip(y_t, Bs):
        likelihoods *= B[:, y]
    prior = x_tm1
    
    posterior_unnormalized = likelihoods * prior

    # possible underflow -> we'll fix it later on
    return posterior_unnormalized / posterior_unnormalized.sum()


In [7]:
observations = [[0, 1], [1, 0], [1, 1], [1, 0]] # (D, not B), (C, B), (C, not B), (W, B)
B = [B_1, B_2]
x_tm1 = x_0
predictions = [x_0]
updates = []

for y_i in observations:
    x_updated = update_mv(x_tm1, y_i, B)
    x_tm1 = predict(x_updated, A)

    updates.append(x_updated)
    predictions.append(x_tm1)

    
predictions, updates

([array([0.5, 0.5]),
  array([0.7, 0.3]),
  array([0.66666667, 0.33333333]),
  array([0.43529412, 0.56470588]),
  array([0.61111111, 0.38888889])],
 [array([1., 0.]),
  array([0.93333333, 0.06666667]),
  array([0.47058824, 0.52941176]),
  array([0.82222222, 0.17777778])])

### Smoothing

Say, we've observed up until timestep $T$ and want to incorporate this additional information into our estimates of previous hidden states.
Mathematically, we'd be interested in finding $p(x_t|Y_{0:T})$ or in words: "given all the information we've observed, what is the distribution over the hidden states at timestep $t$?"

We can derive the form of the smoothed distribution as

$$
    p(x_t|Y) = \overbrace{p(x_t|Y_{0:t})}^{\text{filtered result}} \int p(x_{t+1} | x_t) \frac{\overbrace{p(x_{t+1}|Y)}^{\text{prev. smoothed result}}}{\underbrace{p(x_{t+1}|Y_{0:t})}_{\text{prediction result}}}dx_{t+1}
$$


(For details, please see Phillip Hennigs Lecture "Probabilistic Machine Learning" Lecture 20: Gauss Markov Models)

### Calculation

Lets assume the same setup as in the univariate case before and calculate the smoothened distribution for timestep $t=2$, assuming $p(x_3|Y) = p(x_3|Y_{0:3})$:

\begin{align*}
    p(x_2|Y) &= \overbrace{p(x_2|Y_{0:2})}^{\text{filtered result}} \int p(x_{3} | x_2) \frac{\overbrace{p(x_{3}|Y)}^{\text{prev. smoothed result}}}{\underbrace{p(x_{3}|Y_{0:2})}_{\text{prediction result}}}dx_{3}\\
    &= \begin{bmatrix}
        P(x_2 = S | Y_{0:2})\\
        P(x_2 = R | Y_{0:2})
    \end{bmatrix} \odot \mathcal A \frac{p(x_3|Y)}{p(x_3|Y_{0:2})}\\
    &= \begin{bmatrix}
        P(x_2 = S | Y_{0:2})\\
        P(x_2 = R | Y_{0:2})
    \end{bmatrix} \odot \mathcal A \begin{bmatrix}
        \frac{P(x_3 = S | Y)}{P(x_3 = S | Y_{0:2})}\\
        \frac{P(x_3 = R | Y)}{P(x_3 = R | Y_{0:2})}
    \end{bmatrix}
\end{align*}

where $\odot$ denotes pointwise multiplication

In [8]:
A = np.array([
    [0.7, 0.3],
    [0.2, 0.8]
])

B = np.array([
    [0.5, 0.2, 0.3],
    [0.0, 0.2, 0.8]
])

x_0 = np.array([0.5, 0.5])
observations = [0, 1, 1, 2] # [D, C, C, W]
x_tm1 = x_0
predictions = [x_0]
updates = []


for y_i in observations:
    x_updated = update(x_tm1, y_i, B)
    x_tm1 = predict(x_updated, A)

    updates.append(x_updated)
    predictions.append(x_tm1)

In [9]:
def smooth(t : int, x_tp1 : np.ndarray, A : np.ndarray) -> np.ndarray:
    
    filtered = updates[t]
    predicted = predictions[t + 1]
    u = A @ (x_tp1 / predicted)
    ret = filtered * u

    #print(f"Filtered: {filtered}, Predicted : {predicted}, A@(x_tp1/pred) : {u}, Ret: {ret}")
    return ret

In [10]:
smoothed = [updates[-1]]
x_tp1 = updates[-1]

for t in range(len(observations) - 2, -1, -1):
    # print(f"t: {t}, x_tp1: {x_tp1}")
    x_tp1 = smooth(t, x_tp1, A)
    smoothed.append(x_tp1)

smoothed = list(reversed(smoothed))

In [11]:
for i in range(len(observations)):
    print(f'Updated belief: {updates[i]}, Smoothed belief: {smoothed[i]}')

Updated belief: [1. 0.], Smoothed belief: [1. 0.]
Updated belief: [0.7 0.3], Smoothed belief: [0.65333333 0.34666667]
Updated belief: [0.55 0.45], Smoothed belief: [0.44 0.56]
Updated belief: [0.25333333 0.74666667], Smoothed belief: [0.25333333 0.74666667]


### Multivariate Case:

The current implementation of ```smooth``` would actually work already, we only have to switch back to our multivariate implementation of the update.

In [12]:
A = np.array([
    [0.7, 0.3],
    [0.2, 0.8]
])

B_1 = np.array([
    [0.5, 0.2, 0.3],
    [0.0, 0.2, 0.8]
])

B_2 = np.array([
    [0.6, 0.4],
    [0.1, 0.9]
])

x_0 = np.array([0.5, 0.5])

In [13]:
observations = [[0, 1], [1, 0], [1, 1], [1, 0]] # (D, not B), (C, B), (C, not B), (W, B)
B = [B_1, B_2]
x_tm1 = x_0
predictions = [x_0]
updates = []

for y_i in observations:
    x_updated = update_mv(x_tm1, y_i, B)
    x_tm1 = predict(x_updated, A)

    updates.append(x_updated)
    predictions.append(x_tm1)

In [14]:
smoothed = [updates[-1]]
x_tp1 = updates[-1]

for t in range(len(observations) - 2, -1, -1):
    # print(f"t: {t}, x_tp1: {x_tp1}")
    x_tp1 = smooth(t, x_tp1, A)
    smoothed.append(x_tp1)

smoothed = list(reversed(smoothed))
for i in range(len(observations)):
    print(f'Updated belief: {updates[i]}, Smoothed belief: {smoothed[i]}')

Updated belief: [1. 0.], Smoothed belief: [1. 0.]
Updated belief: [0.93333333 0.06666667], Smoothed belief: [0.93333333 0.06666667]
Updated belief: [0.47058824 0.52941176], Smoothed belief: [0.66666667 0.33333333]
Updated belief: [0.82222222 0.17777778], Smoothed belief: [0.82222222 0.17777778]


### Finding the most likely Hidden-State-Sequence

The **Viterbi**-Algorithm is used to find the most likely state-sequence to have produced the observations. It works by making use of dynamic programming, keeping track of all possible best state sequences, then backtracking to get the best sequence. Here is some latex extracted from my thesis, explaining the steps of the algorithm:

Notation: $b_j(o_t)$ indexes into position $(j, o_t)$ of the emission matrix $\mathcal B$, where $o_t$ is the index of the observation at timestep $t$.

Similarily, $a_{ij}$ indexes into position $(i,j)$ of the transition matrix $\mathcal A$.


\begin{align*}
    & \text{Initialization}:\\
    

    &\begin{align*}
        & \delta_1(i) = \pi_ib_i(o_1), \hspace{1cm} 1 \leq i \leq N\\
        & \psi_1(i) = 0
    \end{align*}{}\\
    
    & \text{Recursion}:\\
    

    &\begin{align*}
        & \delta_t(j) = \max_{1 \leq i \leq N}(\delta_{t-1}(i)a_{ij})b_j(o_t) &1 \leq j \leq N\\
        &&2 \leq t \leq T\\
        & \psi_t(j) = \argmax_{1 \leq i \leq N}(\delta_{t-1}(i)a_{ij}) &1 \leq j \leq N\\
        &&2 \leq t \leq T
    \end{align*}{}\\
    
    & \text{Termination}:\\
    

    &\begin{align*}
        & P^* = \max_{1 \leq i \leq N}(\delta_T(i))\\
        & q_T^* = \argmax_{1 \leq i \leq N}(\delta_T(i))
    \end{align*}{}\\
    
    & \text{Backtracking}:\\
    

    &q_t^* = \psi_{t+1}(q_{t+1}^*), \hspace{0.2cm} t = T-1,T-2,...,1
    
\end{align*}


Below, i'll provide a python implementation.

In [15]:
def viterbi(observations : Sequence[int], 
            A : np.ndarray, 
            B : np.ndarray, 
            pi : np.ndarray) -> Sequence[int]:

    num_observations : int = len(observations)
    num_states : int = len(pi)

    # shape: (num_states, num_observations)
    delta : np.ndarray = np.zeros(shape=(num_states, num_observations))
    psi : np.ndarray = np.zeros_like(delta)


    # initialization
    delta[:, 0] = pi * B[:, observations[0]]

    # recursion/iteration
    for t in range(1, num_observations):
        for j in range(num_states):
            transitions = delta[:, t-1] * A[:, j]
            max_i = np.argmax(transitions)
            psi[j,t] = max_i
            delta[j,t] = np.max(transitions) *\
                  B[j, observations[t]]

    # termination
    q_T : int = np.argmax(delta[:, -1], keepdims=False)
    state_sequence_reversed : Sequence[int] = [q_T]

    q_prime : int = q_T
    for t in range(num_observations - 2, -1, -1):
        q_prime = int(psi[q_prime, t + 1])
        state_sequence_reversed.append(q_prime)

    return list(reversed(state_sequence_reversed))

In [16]:
## lets test our viterbi implementation

A = np.array([
    [0.7, 0.3],
    [0.2, 0.8]
])

B = np.array([
    [0.5, 0.2, 0.3],
    [0.0, 0.2, 0.8]
])

pi = np.array(
    [0.5, 0.5]
)
observations = [0, 1, 1, 2] # [D, C, C, W]

MLE_state_sequence : Sequence[int] = viterbi(observations=observations,
                                             A=A,
                                             B=B,
                                             pi=pi)
MLE_state_sequence

[0, 1, 1, 1]

In [17]:
import itertools

In [18]:
# lets check whether the conditional data likelihood is maximized for the found sequence:

def cond_data_likelihood(states : Sequence[int], observations : Sequence[int], A, B, pi) -> float:

    likelihood = pi[states[0]] * B[states[0], observations[0]]
    for t in range(1, len(states)):
        likelihood *= A[states[t-1], states[t]] * B[states[t], observations[t]]
    
    return likelihood

possible_states = itertools.product([0,1], repeat=len(observations))
likelihoods = [(p, cond_data_likelihood(p, observations, A, B, pi)) for p in possible_states]

In [19]:
sorted_likelihoods = list(sorted(likelihoods, key=lambda t : t[1], reverse=True))
sorted_likelihoods

[((0, 1, 1, 1), 0.0015360000000000005),
 ((0, 0, 1, 1), 0.0013440000000000001),
 ((0, 0, 0, 1), 0.0011759999999999997),
 ((0, 0, 0, 0), 0.0010289999999999997),
 ((0, 1, 0, 1), 0.000144),
 ((0, 1, 1, 0), 0.000144),
 ((0, 0, 1, 0), 0.000126),
 ((0, 1, 0, 0), 0.000126),
 ((1, 0, 0, 0), 0.0),
 ((1, 0, 0, 1), 0.0),
 ((1, 0, 1, 0), 0.0),
 ((1, 0, 1, 1), 0.0),
 ((1, 1, 0, 0), 0.0),
 ((1, 1, 0, 1), 0.0),
 ((1, 1, 1, 0), 0.0),
 ((1, 1, 1, 1), 0.0)]

Seems like we found the right solution.

### Multivariate Viterbi!

In [21]:
def viterbi_mv(observations : Sequence[Sequence[int]], 
            A : np.ndarray, 
            Bs : Sequence[np.ndarray], 
            pi : np.ndarray) -> Sequence[int]:
    """
    Viterbi algorithm for multivariate discrete emissions.

    NOTE: observations are of shape (N, D) where D is the number of emission signals.
    The ordering of these signals MUST correspond to the ordering in the list of emission matrices.

    """
    num_observations : int = len(observations)
    num_states : int = len(pi)

    # shape: (num_states, num_observations)
    delta : np.ndarray = np.zeros(shape=(num_states, num_observations))
    psi : np.ndarray = np.zeros_like(delta)


    # initialization
    delta[:, 0] = pi * np.prod([B[:, observations[0]] for B in Bs])

    # recursion/iteration
    for t in range(1, num_observations):
        for j in range(num_states):
            transitions = delta[:, t-1] * A[:, j]
            max_i = np.argmax(transitions)
            psi[j,t] = max_i
            delta[j,t] = np.max(transitions) * \
                np.prod([B[j, observations[t][i]] for i, B in enumerate(Bs)]) 

    # termination
    q_T : int = np.argmax(delta[:, -1], keepdims=False)
    state_sequence_reversed : Sequence[int] = [q_T]

    q_prime : int = q_T
    for t in range(num_observations - 2, -1, -1):
        q_prime = int(psi[q_prime, t + 1])
        state_sequence_reversed.append(q_prime)

    return list(reversed(state_sequence_reversed))

In [22]:
A = np.array([
    [0.7, 0.3],
    [0.2, 0.8]
])

B_1 = np.array([
    [0.5, 0.2, 0.3],
    [0.0, 0.2, 0.8]
])

B_2 = np.array([
    [0.6, 0.4],
    [0.1, 0.9]
])

pi = np.array([0.5, 0.5])
observations = [[0, 1], [1, 0], [1, 1], [1, 0]] # (D, not B), (C, B), (C, not B), (W, B)
Bs = [B_1, B_2]

MLE_state_sequence : Sequence[int] = viterbi_mv(observations=observations,
                                             A=A,
                                             Bs=Bs,
                                             pi=pi)
MLE_state_sequence

[0, 0, 0, 0]

In [23]:
# check mv case

def cond_data_likelihood_mv(states : Sequence[int], 
                            observations : Sequence[Sequence[int]], 
                            A : np.ndarray, 
                            Bs : Sequence[np.ndarray], 
                            pi : np.ndarray) -> float:

    likelihood = pi[states[0]] * np.prod([B[states[0], observations[0][i]] for i, B in enumerate(Bs)])
    
    for t in range(1, len(states)):
        likelihood *= A[states[t-1], states[t]] * np.prod([B[states[t], observations[t][i]] for i, B in enumerate(Bs)])
    
    return likelihood

possible_states = itertools.product([0,1], repeat=len(observations))
likelihoods = [(p, cond_data_likelihood_mv(p, observations, A, Bs, pi)) for p in possible_states]

In [24]:
sorted_likelihoods = list(sorted(likelihoods, key=lambda t : t[1], reverse=True))
sorted_likelihoods

[((0, 0, 0, 0), 3.95136e-05),
 ((0, 0, 1, 0), 1.0886400000000001e-05),
 ((0, 0, 1, 1), 7.257600000000002e-06),
 ((0, 0, 0, 1), 2.8224000000000007e-06),
 ((0, 1, 1, 0), 2.073600000000001e-06),
 ((0, 1, 1, 1), 1.382400000000001e-06),
 ((0, 1, 0, 0), 8.064000000000003e-07),
 ((0, 1, 0, 1), 5.760000000000004e-08),
 ((1, 0, 0, 0), 0.0),
 ((1, 0, 0, 1), 0.0),
 ((1, 0, 1, 0), 0.0),
 ((1, 0, 1, 1), 0.0),
 ((1, 1, 0, 0), 0.0),
 ((1, 1, 0, 1), 0.0),
 ((1, 1, 1, 0), 0.0),
 ((1, 1, 1, 1), 0.0)]

### A Note on Numerics

The way we've been calculating the probabilities and likelihoods is fine for any small number of states and observations, but lets see what happens, if we want to calculate the conditional data likelihood for a sequence, which is a bit longer...

In [25]:
observations = [0]*1000 # observing only dry weather
states = [0]*1000 # only sunshine
cond_data_likelihood(states = states, observations=observations, A=A, B=B, pi=pi)

0.0

... the likelihood is just so tiny, that python decides it is not worth the precision...

For now we will simply ignore that this is a problem and move on.

#

## Finding $\mathcal A, \mathcal B \text{ and } \pi$

Usually, Hidden Markov Models are modelled as **left-right**, constraining the matrix $\mathcal A$ to a lower triangular matrix or as **fully connected** (no constraints on possible state transitions). The entries in the matrices have to be learned from data using the Expectation-Maximization Algorithm. We'll briefly motivate this algorithm:

Ideally, we'd like to maximize the **marginal (log-) likelihood (evidence)** of our model, i.e. finding $\boldsymbol{\theta} = (\mathcal A, \mathcal B \text{ and } \pi)$ that **maximize** the likelihood making our observations. In mathematical terms, we'd like to maximize
$$
    \log p(Y|\boldsymbol{\theta}) = \log \int p(Y, X | \boldsymbol{\theta} )dX
$$

(maximizing the log-likelihood yields the same result, as the logarithm is a monotonic function which preserves the location of extrema)

This however can be very hard... :( What do instead is compute an Expectation of the "complete data" log likelihood for any fixed $\boldsymbol{\theta}^*$

$$
    q(\boldsymbol{\theta}, \boldsymbol{\theta}^*) = \int p(X|Y, \boldsymbol{\theta}^*) \log p(X, Y| \boldsymbol{\theta}) dX
$$

and then maximize w.r.t. $\boldsymbol{\theta}$. This can be easier, as the $\log$ may simplify things.

In hidden markov models, there exists an algorithm called 'Baum-Welch', which does exactly this. It iteratively updates model hyperparameters until convergence. The derivation is a bit involved, we'd start by noting, that for our discrete case,

\begin{align*}
    q(\theta, \theta^*) &= \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \log p(X, Y|\theta)\\
    &= \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \log \left(\pi_\theta \mathcal B _\theta[x_1, y_1] \prod_{i = 2}^T \mathcal A_\theta [x_{i-1}, x_i]\mathcal B _\theta[x_i, y_i]\right)\\
    &= \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \log \pi_\theta \\
    &+ \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \sum_{i=2}^T\log \mathcal A_\theta[x_{i-1}, x_{i}]\\
    &+ \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \sum_{i=1}^T\log \mathcal B_\theta[x_i, y_i]\\
\end{align*}

which we may then optimize seperately for $\pi, \mathcal A, \mathcal B$, making use of Lagrange multipliers to constrain the result to represent a valid probability distribution.

For the univariate case, the optimization problem, including Lagrange multipliers would look like this:

\begin{align*}
    \Lambda(\boldsymbol \theta, \boldsymbol \theta^*) &= q(\boldsymbol{\theta}, \boldsymbol \theta^*) - \lambda_\pi (\sum_i^N \pi_i - 1) - \sum_i^N \lambda_{A_i} (\sum_j^N A_{ij} - 1) \\
    &- \sum_i^N \lambda_{B_i} (\sum_j^M B_{ij} - 1)
\end{align*}

For the multivariate emission case, we have to add constraints for the additional observation matrices $- \sum_k^K\sum_i^{N} \lambda_{B^k_{i}} (\sum_j^{M_k} B_{ij}^k - 1)$. 

### An annotated derivation

To find find closed form updates for the parameters $\pi, \mathcal A$ and $\mathcal B$, we have to partially differentiate $\Lambda(\theta, \theta*)$. We consider $D$ sequences of observations $Y^{(d)} = y_1^{(d)}, ..., y_N^{(d)}$.

First, lets derive the update for $\pi_i$.

\begin{align*}
\frac{\partial \Lambda(\theta, \theta^*)}{\partial \pi_i} &= \frac{\partial}{\partial \pi_i}\left( \sum_{X \in \mathcal X} \sum_{d=1}^D \log \pi_{X^{(d)}_1} P(X, Y|\theta^*)\right) - \lambda_\pi = 0\\
    &= \frac{\partial}{\partial \pi_i}\left( \sum_{X_1}\sum_{X_2} \dots \sum_{X_T} \sum_{d=1}^D \log \pi_{X^{(d)}_1} P(X^{(d)}_1, X^{(d)}_2, \dots, X^{(d)}_T, Y|\theta^*)\right) - \lambda_\pi = 0\\
    &\overset{(1)}{=} \frac{\partial}{\partial \pi_i}\left( \sum_{j = 1}^N \sum_{d=1}^D \log \pi_j P(X^{(d)}_1 = j, Y|\theta^*)\right) - \lambda_\pi = 0\\
    &\overset{(2)}{=}  \sum_{d=1}^D \frac{P(X^{(d)}_1 = i, Y|\theta^*)}{\pi_i} - \lambda_\pi = 0\\
    \frac{\partial \Lambda(\theta, \theta^*)}{\partial \lambda_\pi} &= (\sum_{i=1}^N \pi_i) - 1 = 0
\end{align*}

At $(1)$, we marginalize out all Random Variables $X_t$ with timesteps $t \neq 1$, leaving only the states for timestep $t = 1$, which there are $N$ of. Additionally, we marginalize out all $X^{(d' \neq d)}_t$ for each $d$.

At $(2)$, we differentiate w.r.t. $\pi_i$, which only leaves entries, for which $X^{(d)}_1 = i$.

From there on, we solve the homogenous system of equations.

\begin{bmatrix}
    \sum_{d=1}^D \frac{P(X^{(d)}_1 = 1, Y|\theta^*)}{\pi_1} - \lambda_\pi &= 0\\
    \sum_{d=1}^D \frac{P(X^{(d)}_1 = 2, Y|\theta^*)}{\pi_2} - \lambda_\pi &= 0\\
    \vdots & \vdots \\
    \sum_{d=1}^D \frac{P(X^{(d)}_1 = N, Y|\theta^*)}{\pi_N} - \lambda_\pi &= 0\\
    \sum_{i=1}^N \pi_i &= 1
\end{bmatrix}

which yields the solution

\begin{align*}
    \pi_i &= \frac{\sum_{d=1}^D P(X^{(d)}_1 = i, Y|\theta^*)}{\sum_{j = 1}^N \sum_{d=1}^D P(X^{(d)}_1 = j, Y|\theta^*)}\\
    &= \frac{\sum_{d=1}^D P(X^{(d)}_1 = i, Y|\theta^*)}{\sum_{d=1}^D \sum_{j = 1}^N P(X^{(d)}_1 = j, Y|\theta^*)}\\
    &= \frac{\sum_{d=1}^D P(X^{(d)}_1 = i, Y|\theta^*)}{\sum_{d=1}^D P(Y|\theta^*)}\\
    &= \frac{\sum_{d=1}^D P(X^{(d)}_1 = i, Y|\theta^*)}{D P(Y|\theta^*)}\\
    &= \frac{\sum_{d=1}^D P(X^{(d)}_1 = i|Y, \theta^*) P(Y|\theta^*)}{D P(Y|\theta^*)}\\
    &= \frac{1}{D} \sum_{d=1}^D P(X^{(d)}_1 = i|Y^{(d)}, \theta^*)
\end{align*}

For our updates to $\mathcal A$, lets examine the partial derivative w.r.t. $A_{ij}$.

\begin{align*}
    \frac{\partial \Lambda(\theta, \theta^*)}{\partial A_{ij}} &= \frac{\partial}{\partial A_{ij}} \left( \sum_{X \in \mathcal X} \sum_{d=1}^D \sum_{t=2}^T \log A_{X^{(d)}_{t-1}, X^{(d)}_{t}} P(X, Y; \theta^*)\right) - \lambda_{A_i} = 0\\
    &\overset{(1)}{=} \frac{\partial}{\partial A_{ij}} \left( \sum_{j = 1}^N \sum_{k = 1}^N \sum_{d=1}^D \sum_{t=2}^T \log A_{jk} P(X^{(d)}_{t-1} = j, X^{(d)}_{t} = k, Y; \theta^*)\right) - \lambda_{A_i} = 0\\
    &= \sum_{d=1}^D \sum_{t=2}^T \frac{P(X^{(d)}_{t-1} = i, X^{(d)}_{t} = j, Y; \theta^*)}{A_{ij}} - \lambda_{A_i} = 0\\
    \frac{\partial \Lambda(\theta, \theta^*)}{\partial \lambda_{A_{i}}} &= (\sum_{j=1}^N A_{ij}) - 1 = 0
\end{align*}

where at $(1)$, we again have marginalized out alot of Random Variables, which simplifies the equation. Solving the system of equations, we end up with the update:

\begin{align*}
    A_{ij} &= \frac{\sum_{d=1}^D \sum_{t=2}^T P(X^{(d)}_{t-1} = i, X^{(d)}_{t} = j, Y; \theta^*)}{\sum_{j=1}^N \sum_{d=1}^D \sum_{t=2}^T P(X^{(d)}_{t-1} = i, X^{(d)}_{t} = j, Y; \theta^*)}\\
    &\overset{(*)}{=} \frac{\sum_{d=1}^D \sum_{t=2}^T P(X^{(d)}_{t-1} = i, X^{(d)}_{t} = j, Y^{(d)}; \theta^*)}{\sum_{d=1}^D \sum_{t=2}^T P(X^{(d)}_{t-1} = i, Y^{(d)}; \theta^*)}\\ 
\end{align*}

$(*)$ details: https://stephentu.github.io/writeups/hmm-baum-welch-derivation.pdf

For $\mathcal B$ the derivation is similar, but we use an indicatorfunction 

$$
I(Y_t^{(d)} = k) = \begin{cases} 1 & Y_t^{(d)} = k\text{ evaluates to True}\\
0 & \text{otherwise}\end{cases}
$$

in the computation of the partial derivative, because we cannot preemtively evaluate our $ Y^{(d)}_t$ s. Note, for reasons of convention, we'll use $B_i(j)$ to denote the probability of emitting observation $Y^{(d)} = j$ under state $i$.
We have:

\begin{align*}
    \frac{\partial \Lambda(\theta, \theta^*)}{\partial B_i(j)} &= \frac{\partial}{\partial B_i(j)} \left( \sum_{X \in \mathcal X}\sum_{d=1}^D \sum_{t=1}^T \log B_{X_t^{(d)}}(Y_t^{(d)}) P(X, Y|\theta^*)\right) - \lambda_{B_i} = 0\\
    &\overset{(1)}{=} \frac{\partial}{\partial B_i(j)} \left( \sum_{i = 1}^M \sum_{d=1}^D \sum_{t=1}^T \log B_i(Y_t^{(d)}) P(X_t^{(d)} = i, Y|\theta^*)\right) - \lambda_{B_i} = 0\\
    &\overset{(2)}{=} \sum_{d=1}^D \sum_{t=1}^T \frac{P(X_t^{(d)} = i, Y| \theta^*)I(Y_t^{(d)} = j)}{B_i(j)} - \lambda_{B_i} = 0\\
    \frac{\partial \Lambda(\theta, \theta^*)}{\partial \lambda_{B_i}} &= \sum_{j=1}^N B_i(j) - 1 = 0
\end{align*}

where at $(1)$ we marginalize, and at $(2)$ we use the indicator function to correctly compute the derivative. This we have to do, as the $Y_t^{(d)}$ s are already observed and an update on the entry $B_{ij}$ depends on whether or not we actually have observed any $Y_t^{(d)} = j$.

We end up with the following form for the update:

\begin{align*}
    B_i(j) &= \frac{\sum_{d=1}^D \sum_{t=1}^T P(X_t^{(d)} = i, Y|\theta^*)I(Y_t^{(d)} = j)}{\sum_{j = 1}^M \sum_{d=1}^D \sum_{t=1}^T P(X_t^{(d)} = i, Y|\theta^*)I(Y_t^{(d)} = j)}\\
    &= \frac{\sum_{d=1}^D \sum_{t=1}^T P(X_t^{(d)} = i, Y|\theta^*)I(Y_t^{(d)} = j)}{ \sum_{d=1}^D \sum_{t=1}^T P(X_t^{(d)} = i, Y|\theta^*)}\\
    &= \frac{\sum_{d=1}^D \sum_{t=1}^T P(X_t^{(d)} = i | Y^{(d)},\theta^*)I(Y_t^{(d)} = j)}{ \sum_{d=1}^D \sum_{t=1}^T P(X_t^{(d)} = i | Y^{(d)}, \theta^*)}\\
\end{align*}

### Learning multiple $\mathcal B$ s simultaniously

Going back to our multivariate example, where the likelihood is the product of categorical likelihoods, we have to learn $K$ emission matrices $\mathcal B^k$ at once. Lets see how the hyperparameter update equations change:

\begin{align*}
    q(\theta, \theta^*) &= \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \log p(X, Y|\theta)\\
    &= \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \log \left(\pi_\theta  \prod_{k=1}^K \mathcal B^k _\theta[x_1, y_1] \prod_{i = 2}^T\left( \mathcal A_\theta [x_{i-1}, x_i]\prod_{k=1}^K \mathcal B^k_\theta[x_i, y^k_i]\right)\right)\\
    &= \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \log \pi_\theta \\
    &+ \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \sum_{i=2}^T\log \mathcal A_\theta[x_{i-1}, x_{i}]\\
    &+ \sum_{k = 1}^K \sum_{X \in \boldsymbol{X}} p(X|Y, \theta^*) \sum_{i=1}^T\log \mathcal B^k_\theta[x_i, y^k_i]\\
\end{align*}


The update equations for $\pi, \mathcal A$ don't change dramatically, as we are able to compute $P(X^{(d)}_1 = i|Y^{k, (d)}, \theta^*)$ and $P(X^{(d)}_{t-1} = i, X^{(d)}_{t} = j |Y^{k, (d)}, \theta^*)$. Hence the math works out the same. The updates for the emission matrices can be computed **seperately**, giving us the same update equation.